In [1]:
import torch
from torch.utils.data import DataLoader

import pandas as pd

from FLTrack.models import ShallowNN
from FLTrack.evals import evaluate_mae_with_confidence

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [2]:
selected_client_ids_param = ["0_0", "0_2", "0_3", "0_4", "1_0", "1_1", "1_2", "1_4", "2_0", "2_1", "2_2", "2_3", "2_4", "2_5", "3_0", "3_2","3_3","3_4","3_5"]
selected_client_ids_glob = client_ids = ["0_0","0_1","0_5","1_0","1_1","1_4","1_5","2_0","2_1","2_2","2_3","2_4","2_5","3_0","3_1","3_2","3_3","3_4","3_5",]
client_ids = [f"{i}_{j}" for i in range(4) for j in range(6)]

In [3]:
selected_global_model_param = ShallowNN(features)
selected_global_model_param.load_state_dict(torch.load('checkpt/saving_only_selected_clients/epoch_500/_fedl_global_20_25.pth'))

global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/saving/epoch_500/_fedl_global_20_25.pth'))

selected_global_model_glob = ShallowNN(features)
selected_global_model_glob.load_state_dict(torch.load('checkpt/saving_only_selected_clients_g/epoch_500/_fedl_global_20_25.pth'))

<All keys matched successfully>

In [4]:
selected_eval_list_p = []
for client in selected_client_ids_param:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(selected_global_model_param,val_loader)
    
    eval_dict = {"Client ID" : client,
                "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    
    selected_eval_list_p.append(eval_dict)
    
selected_eval_df = pd.DataFrame.from_dict(selected_eval_list_p)
selected_eval_df["clients"] = [i for i in range(1, 25) if i not in (2, 6, 10, 12, 20)]

In [5]:
selected_eval_list_g = []
for client in selected_client_ids_glob:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(selected_global_model_glob,val_loader)
    
    eval_dict = {"Client ID" : client,
                "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    
    selected_eval_list_g.append(eval_dict)
    
selected_eval_df_g = pd.DataFrame.from_dict(selected_eval_list_g)
selected_eval_df_g["clients"] = [i for i in range(1, 25) if i not in (2, 6, 10, 12, 20)]

In [6]:
eval_list = []
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(global_model,val_loader)
    
    eval_dict = {"Client ID" : client,
                "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    
    eval_list.append(eval_dict)
    
eval_df = pd.DataFrame.from_dict(eval_list)
eval_df["clients"] = [i for i in range(1, 25)]

In [18]:
for client in selected_client_ids_param:
    selected_model_mae = selected_eval_df.loc[selected_eval_df['Client ID'] == client]["Federated Average MAE"].values[0]
    all_model_mae = eval_df.loc[eval_df['Client ID'] == client]["Federated Average MAE"].values[0]
    print(client, selected_model_mae, all_model_mae)

0_0 0.4954 0.4796
0_2 0.3337 0.3645
0_3 0.4221 0.4637
0_4 0.3335 0.3595
1_0 0.506 0.4997
1_1 0.5111 0.5001
1_2 0.4223 0.4418
1_4 0.4415 0.4443
2_0 0.4211 0.4227
2_1 0.4469 0.4288
2_2 0.424 0.4244
2_3 0.4248 0.4323
2_4 0.4164 0.4194
2_5 0.4268 0.4358
3_0 0.5 0.5187
3_2 0.5136 0.5243
3_3 0.5035 0.5187
3_4 0.509 0.5215
3_5 0.5153 0.5307


In [9]:
for client in selected_client_ids_glob:
    selected_model_mae_g = selected_eval_df_g.loc[selected_eval_df_g['Client ID'] == client]["Federated Average MAE"].values[0]
    all_model_mae = selected_eval_df_g.loc[selected_eval_df_g['Client ID'] == client]["Federated Average MAE"].values[0]
    print(client, selected_model_mae, all_model_mae)

0_0 0.5153 0.4955
0_1 0.5153 0.4978
0_5 0.5153 0.4702
1_0 0.5153 0.503
1_1 0.5153 0.5065
1_4 0.5153 0.4864
1_5 0.5153 0.5778
2_0 0.5153 0.4277
2_1 0.5153 0.4524
2_2 0.5153 0.4217
2_3 0.5153 0.4228
2_4 0.5153 0.4154
2_5 0.5153 0.4248
3_0 0.5153 0.5064
3_1 0.5153 0.5072
3_2 0.5153 0.5152
3_3 0.5153 0.5012
3_4 0.5153 0.5162
3_5 0.5153 0.5196


In [14]:
selected_model_average_mae_param = selected_eval_df["Federated Average MAE"].mean()
selected_model_average_mae_global = selected_eval_df_g["Federated Average MAE"].mean()
model_average_mae = eval_df["Federated Average MAE"].mean()

In [15]:
print(f"Average mae of selected model by param {selected_model_average_mae_param}")
print(f"Average mae of selected model by global {selected_model_average_mae_global}")
print(f"Average mae of model {model_average_mae}")

Average mae of selected model by param 0.45089471340179443
Average mae of selected model by global 0.48251578211784363
Average mae of model 0.47159168124198914
